In [500]:
import requests
from selenium.webdriver import (Chrome, Firefox, ChromeOptions, FirefoxProfile)
import pymongo
import datetime
from bs4 import BeautifulSoup
import pandas as pd
import yaml
import time
import random

In [501]:
# Variables
path_to_fb_login_yaml = "/Users/jasonli/.secrets/facebook-web-scrape-cred.yaml"
my_profile_url = "https://www.facebook.com/jason.li.96930"

In [502]:
# Initialize MongoDB
mc = pymongo.MongoClient()
db = mc['my-facebook-webscrape']
fb_statuses = db['fb-statuses']

In [553]:
# Read FB Credentials
with open(path_to_fb_login_yaml, 'r') as stream:
    try:
        y = yaml.load(stream)
        my_password = y['password']
        my_email = y['email']
    except yaml.YAMLError as exc:
        print(exc)

In [605]:
# Choose a Browser:

# CHROME
# options = ChromeOptions();
# options.add_argument("--disable-notifications");

# browser = Chrome(options=options)

# FIREFOX
profile = FirefoxProfile();
profile.set_preference("dom.webnotifications.enabled", False);
browser = Firefox(firefox_profile=profile)

In [ ]:
# Login to FB in Selenium Browser
url = 'https://www.facebook.com/'
browser.get(url)

email = browser.find_element_by_id('email')
password = browser.find_element_by_id('pass')

email.send_keys(my_email)
password.send_keys(my_password)

browser.find_element_by_id("loginbutton").click()

In [580]:
# # browser.get('https://www.facebook.com/eDDIEBOYMANG')
# posts = browser.find_elements_by_css_selector('div[id*=tl_unit]')
# print('posts: ' + str(len(posts)))
# for post in posts:
#     post_time_element = post.find_element_by_css_selector('abbr')
#     post_time = post_time_element.get_attribute('title')

#     user_content_element = post.find_element_by_css_selector('div[class*=userContent]')
    
#     post_context = post.find_element_by_css_selector('h5')
    
#     if ('Eddie Ignacio' in post_context.text and
#         'is with' not in post_context.text and
#         'was tagged' not in post_context.text and
#         'is in' not in post_context.text and
#         'to' not in post_context.text): 
#         print(post_context.text)
#         para_elements = user_content_element.find_elements_by_css_selector('p')
#         for p in para_elements:
#             print(p.text)

posts: 65
Eddie Ignacio shared a video.
Time lapse from dash cam of the wall of lava consuming the Ford Mustang.
Eddie Ignacio shared a link.
Eddie Ignacio shared a video.
In the 1990s, Street Fighter II ruled the world, and no one did it better than teenager Tomo Ohira. But after solidifying himself as a gaming legend, Tomo disappeared ...
Join us as we go on a mission to track down Tomo in the Emmy® nominated series 8 Bit Legacy.
Eddie Ignacio shared a memory.
Kinda regret selling this thing
For sale 1969 Datsun 510 wagon rebuilt engine <5k miles, brand new weber, complete interior runs strong $5,500 #datsun #ratsun #510 #datsun510
Eddie Ignacio shared a link.
Eddie Ignacio shared a post.

Pineapple
Eddie Ignacio
I use to shop here when I lived on Eastlake across the bridge from u district
Eddie Ignacio shared a link.
Eddie Ignacio shared a memory.
Kony Kitchen
Nic Rutten
Kony kitchen 2012
Eddie Ignacio shared a video.
I KNEW IT 🔍
[via zero_fcksgirl]
Eddie Ignacio
My ears are frozen 

In [508]:
# Navigate to the friends tab in your profile
browser.find_element_by_css_selector(f'a[href="{my_profile_url}"]').click()
time.sleep(2)

browser.find_element_by_css_selector('a[data-tab-key="friends"]').click()
time.sleep(2)

number_of_friends = int(browser.find_element_by_name('All Friends').find_elements_by_css_selector('span')[1].text)
print('Number of friends: ' + str(number_of_friends))

416


In [296]:
# Create a dictionary of friends and profile_urls,
# where key=friend_name and value=friend_profile_url 
SCROLL_PAUSE_TIME = 1

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

friends_profiles_dict = {}

while len(friends_profiles_dict.items()) < number_of_friends:

    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)
    
    
    friend_items = browser.find_elements_by_css_selector('div[data-testid=friend_list_item]')
    
    if len(friends_profiles_dict.items()) > number_of_friends:
        break
    
    for friend_item in friend_items:
        profile_links = friend_item.find_elements_by_css_selector('a')
            
        for profile in profile_links:
            url = profile.get_attribute('href')
            
            if (
                my_profile_url not in url and 
                '?' in url and 
                'browse/mutual_friends/' not in url
            ):
                if not any(char.isdigit() for char in profile.text) or not profile.text:
                    if 'profile.php?id=' in url:
                        friends_profiles_dict[profile.text] = url
                    else:
                        s = url.split('?')[0]
                        friends_profiles_dict[profile.text] = s
            
    print('Creating friends dictionary... \n Friend count: ' +  str(len(friends_profiles_dict.items())) + ' friends.')

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        print('Finished creating friends dictionary!')
        break
    last_height = new_height

21
41
81
101
120
139
158
175
194
214
234
254
254
293
313
332
352
372
391
402
402
402
402
402
402
402
402


In [590]:
# Remove blank values
friends_profiles_dict = {k: v for k, v in friends_profiles_dict.items() if k is not ''}
friends_profiles_dict

{'Adam Koehler': 'https://www.facebook.com/adam.koehler.50',
 'Adanimo Nrg': 'https://www.facebook.com/adam.arguello.35',
 'Adelina Manaut': 'https://www.facebook.com/adelinamanaut',
 'Alana McKnight': 'https://www.facebook.com/alanajoy93',
 'Alannah Rose': 'https://www.facebook.com/alannahnahnah',
 'Alex Beaton': 'https://www.facebook.com/alex.beaton.988',
 'Alex Chang': 'https://www.facebook.com/changAlang24',
 'Alex Grob': 'https://www.facebook.com/Grob.Alex',
 'Alex Keeno': 'https://www.facebook.com/akeeno11',
 'Alexander Cheung': 'https://www.facebook.com/alexander.cheung.564',
 'Alexander Hooton': 'https://www.facebook.com/alexander.hooton.5',
 'Alexander Pang': 'https://www.facebook.com/alexander.pang.5',
 'Alexander Robinson': 'https://www.facebook.com/PRIDE908',
 'Alexis Szigeti': 'https://www.facebook.com/alexismarieszigeti',
 'Alfred Quaicoe': 'https://www.facebook.com/fred.amissah',
 'Ali Patton': 'https://www.facebook.com/ali.patton.12',
 'Ally Holtz': 'https://www.faceboo

In [ ]:
already_scraped = {
    'Kellian Valenti': 'https://www.facebook.com/kpv093',
    'Chen Chi': 'https://www.facebook.com/Melodycc55',
    'Jonathan Cheung': 'https://www.facebook.com/jonathan.cheung.545',
    'Sybil Cheng': 'https://www.facebook.com/sybil.cheng1',
}

In [ ]:
friends_to_scrape_keys = set(friends_profiles_dict.keys()) - set(already_scraped.keys())

friends_to_scrape = {}
for key in friends_to_scrape_keys:
    friends_to_scrape[key] = friends_profiles_dict[key]

In [ ]:
# Web scrape each friend in friends dictionary, add statuses to mongoDB.
# Data structure of each entry:
#   {'name': STRING = name,
#   'url': STRING = profile url,
#   'datetime': DATETIME = current time,
#   'statuses': DICT = {key=time of status post, value=status}, 
#   'html': STRING = html of page,}

number_of_statuses = 200

min_scroll_time = 3

# Iterate through each friend in the friends dictionary
for name, url in mod_friends_profiles_dict.items():
    person_dict = fb_statuses.find_one({"url": url})
    
#     if person not in DB
    if person_dict == None:
        statuses_dict = {}
    else:
        statuses_dict = person_dict['statuses']
    
    browser.get(url)

    # Get scroll height
    last_height = browser.execute_script("return document.body.scrollHeight")

    
    # Scroll through friends timeline and add statuses to dictionary    
    while len(statuses_dict.items()) < number_of_statuses:

        SCROLL_PAUSE_TIME = min_scroll_time * (1 + random.random())

        # Scroll down to bottom
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        
        posts = browser.find_elements_by_css_selector('div[id*=tl_unit]')
        
        for post in posts:
            try:
                post_time_element = post.find_element_by_css_selector('abbr')
                post_time = post_time_element.get_attribute('title')
                post_context = post.find_element_by_css_selector('h5')
            
                # Conditionals to weed out non authored posts
                if (post_time not in statuses_dict.keys() and
                    name in post_context.text and
                    'is with' not in post_context.text and
                    'was tagged' not in post_context.text and
                    'is in' not in post_context.text and
                    'to' not in post_context.text): 

                    user_content_element = post.find_element_by_css_selector('div[class*=userContent]')
                    para_elements = user_content_element.find_elements_by_css_selector('p')

                    # Status sometimes split in two p elements. Merge two paragraphs
                    if len(para_elements) > 0:
                        text = ''
                        for para_element in para_elements:
                            text += para_element.text + ' '

                    print('Time: ' + post_time + '\n' + 'Status: ' + text + '\n')

                    # Add status to dictionary
                    statuses_dict[post_time] = text
            except:
                pass
        print('Creating statuses dictionary for ' + name + '... \n' + 'Status count: ' +  str(len(statuses_dict.items())) + ' statuses.')
        

        # Calculate new scroll height and compare with last scroll height
        new_height = browser.execute_script("return document.body.scrollHeight")
        
        # Reached the end of friend's timeline, add name to already_scraped_dict
        if new_height == last_height:
            print('Finished creating ' + name + ' statuses dictionary!')
            already_scraped_dict[name] = url
            break
        last_height = new_height
        
    html = browser.page_source
    
    # Add entry to MongoDB
    fb_statuses.update_one(
        {'url': url}, 
        {'$set': {
            'name': name,
            'url': url,
            'datetime': datetime.datetime.now(),
            'statuses': statuses_dict, 
            'html': html, 
            }
        },
    upsert=True
    )

Time: 12/15/2016 11:08am
Status: Wow does time fly! Miss my Cohort! Best cohort hands down! Love you guys and I mean that. 

Creating statuses dictionary for Sybil Cheng... 
Status count: 1 statuses.
Creating statuses dictionary for Sybil Cheng... 
Status count: 1 statuses.
Creating statuses dictionary for Sybil Cheng... 
Status count: 1 statuses.
Creating statuses dictionary for Sybil Cheng... 
Status count: 1 statuses.
Time: 08/14/2015 5:57pm
Status: Must watch awesome chemical reactions.. don't miss it at any chance! 

Time: 02/28/2015 12:50pm
Status: Must watch awesome chemical reactions.. don't miss it at any chance! 

Creating statuses dictionary for Sybil Cheng... 
Status count: 3 statuses.
Creating statuses dictionary for Sybil Cheng... 
Status count: 3 statuses.
Creating statuses dictionary for Sybil Cheng... 
Status count: 3 statuses.
Time: 06/09/2014 5:47pm
Status: Must watch awesome chemical reactions.. don't miss it at any chance! 

Creating statuses dictionary for Sybil C

Creating statuses dictionary for Sybil Cheng... 
Status count: 35 statuses.
Time: 05/21/2009 9:55am
Status: Is waiting to board her flight to LA. The airport is PACKED. 

Time: 05/18/2009 7:15pm
Status: Should join geni.com 

Creating statuses dictionary for Sybil Cheng... 
Status count: 37 statuses.
Time: 05/08/2009 8:14am
Status: Is waiting for workplace to open the doors. This inconsistant schedule is bugging me. 

Time: 05/02/2009 8:54pm
Status: is about to watch Wolverine Origins with Jimmy and Ed. 

Time: 04/22/2009 1:14pm
Status: is about to watch Wolverine Origins with Jimmy and Ed. 

Time: 04/09/2009 2:28pm
Status: is still feeling some jet lag 

Time: 04/09/2009 11:06am
Status: is looking for something to do until 11:30. 

Time: 03/23/2009 11:34pm
Status: is waiting to board her Taiwan flight. 

Creating statuses dictionary for Sybil Cheng... 
Status count: 43 statuses.
Time: 02/21/2009 6:47am
Status: Is headed up to the slopes. 

Time: 02/07/2009 1:19pm
Status: will be in th

Creating statuses dictionary for Annelise Yee... 
Status count: 10 statuses.
Time: 09/17/2016 7:40pm
Status: Literally my life right now 

Time: 09/16/2016 5:01pm
Status: CAN'T WAIT Watch the brand new trailer for Disney’s Moana! See the film, starring Dwayne The Rock Johnson & Auli'i Cravalho, in theatres in 3D this Thanksgiving! 

Creating statuses dictionary for Annelise Yee... 
Status count: 12 statuses.
Time: 07/07/2016 7:04am
Status: 575732_10150931419207921_1333108867_n 

Time: 06/15/2016 1:07pm
Status: So excited for this! https://www.facebook.com/disneymoana/videos/1722989297941995/?pnref=story.unseen-section Watch the brand new teaser trailer for Disney’s Moana! See the film, starring Dwayne The Rock Johnson & Auli'i Cravalho, in theatres this Thanksgiving! 

Time: 06/14/2016 12:04pm
Status: Somehow, I managed to survive RISD Professor Clara Lieu’s insanely challenging drawing class and emerged as a much stronger artist. Now, I’m working with her on ART PROF: a free, online e

Time: 11/11/2013 5:34pm
Status: really really really hope it snows tomorrow (: I might regret saying that but I really can't wait (: 

Time: 11/09/2013 12:46pm
Status: <3 

Time: 10/30/2013 8:00pm
Status: Just spilt coffee on 5.5 hours worth of my drawing thats due friday... loving life 

Creating statuses dictionary for Annelise Yee... 
Status count: 55 statuses.
Time: 10/07/2013 11:37pm
Status: Crazy throw back! Youngin days 

Time: 10/02/2013 6:35pm
Status: 1 play to read, 2 papers to write, a drawing to draw and 3 cups of coffee, gonna be a longggg night 

Time: 09/25/2013 12:39pm
Status: Having a Brother Iz jam sesh right now... craving some home 

Creating statuses dictionary for Annelise Yee... 
Status count: 58 statuses.
Time: 09/21/2013 11:12am
Status: 2 papers, 2 readings, 7.5 more drawings, a straw bridge and a portfolio to design ... This weekend looks promisingly fun... 

Time: 09/12/2013 10:47pm
Status: My first east coast thunderstorm (: I'm lovin it 

Time: 09/11/2013 8

{'04/07/2018 10:17pm': 'I am now drinking Dark Souls while I play alcohol: www.twitch.tv/t7chicken Come watch me get rekt! ',
 '04/07/2018 9:00pm': "Sad Reacts only! 'I spent an hour crying over a goose just for this to happen' 😂😂 ",
 '04/08/2018 11:49pm': "Pariss Athena These kids are going nuts lol This'll be the last time Dad dresses as the Easter Bunny... 😳😂 ",
 '04/08/2018 7:42pm': 'I asked my bro to clip my toe nails for money. Me: 10 bucks Him: 20        ',
 '04/09/2018 9:09am': 'Ddaayyyuumm 😱😱😱 ***READY FOR SHOWINGS***\nFort Worth,TX 76126 | 9553 Bella Terra Dr.\nReady For Showings $7,000,000\n"PLEASE SHARE"           ',
 '04/10/2018 7:00am': "OMG this is so meta. I'm in tears lmao Positively Debordian. ",
 '04/10/2018 7:10pm': 'Nick DeJesus This one tho ',
 '04/11/2018 11:52pm': 'When I tell doctors I enjoy cleaning my ears with Q-Tips https://m.imgur.com/B5TBhZr ',
 '04/11/2018 4:32pm': '"how are you still alive?" Once in a blue moon 😂 ',
 '04/11/2018 9:44am': 'Nick DeJesus😂 

In [593]:
fb_statuses.find_one()

{'_id': ObjectId('5af34452ebd4d7fef64d24da'),
 'datetime': datetime.datetime(2018, 5, 9, 11, 56, 18, 385000),
 'html': '<html id="facebook" class="sidebarMode" lang="en"><head><meta charset="utf-8"><meta name="referrer" content="origin-when-crossorigin" id="meta_referrer"><script>window._cstart=+new Date();</script>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [474]:
list(fb_statuses.aggregate([
    {"$group" : { "_id": "$name", "count": { "$sum": 1 } } },
    {"$match": {"_id" :{ "$ne" : None } , "count" : {"$gt": 1} } }] ))
#     {"$sort": {"count" : -1} },
#     {"$project": {"name" : "$_id", "_id" : 0} }     
# )

[{'_id': 'Bradley Li', 'count': 2}]

{'02/08/2018 10:32pm': "*cough cough hint hint*\nYour weekly dose of interesting posts brought to you by me and Amy Isn't It True 😂 ( via Edinburgh Zoo ) ",
 '03/11/2016 3:26pm': "Not sure if I want this game. It looks fun but it doesn't look like it will fill my fps void. Your watch is about to begin—Overwatch arrives May 24! Are you with us? blizz.ly/BuyOverwatch ",
 '06/01/2015 5:02am': '"Here\'s to those olden days,\nHere\'s to those golden days,\nHere\'s to the friends we made at dear old RPI." Gross, I can\'t believe I just quoted our alma mater. Anyways, thanks for making the last 4 years pretty great, BARH Settlement. Google Maps keeps asking me to set 16th St as "Home", and I\'m inclined to click yes. But we kind of moved out. So maybe not. Also, I\'m a =π€®°^¥. Ahem. I\'m a human. My tail didn\'t just slip.  ',
 '06/26/2015 12:51am': 'Noodles no back, bubble shield... even bubble shield on the trash can :P Ha! Even in Taiwan! ',
 '11/08/2015 11:26am': 'Hello!',
 '11/08/2016 3

1

{'01/01/2014 3:04am': 'HAPPY NEW YEARS!! Pop pop pop...man I miss doing fireworks ',
 '01/01/2015 3:01am': 'Happy new years ',
 '01/01/2016 3:09am': "Happy new years 2015 was OK let's hope 2016 will be even better ",
 '01/01/2017 8:23am': 'Happy new years I traveled into the future to deliver this message ',
 '01/08/2015 9:59pm': 'Shaboo shaboo ',
 '01/09/2016 12:43am': 'Close game that kick though ',
 '01/11/2014 7:11pm': 'getting wisdoms pulled sucks ',
 '01/11/2016 10:23pm': 'Close game that kick though ',
 '01/13/2015 9:55pm': 'So.....who is still here? ',
 '01/13/2018 8:51pm': 'I miss all my ads, now I see the feeds of actual people :( ',
 '01/14/2018 5:50pm': 'Happy birthday old man ',
 '01/16/2017 2:50pm': 'Happy MLK jr day ',
 '01/18/2014 2:56pm': 'Back to freezing my ass off 24/7 ',
 '01/19/2014 12:35pm': 'sucks that I have to cook as soon as I get back just because of my teeth ',
 '01/20/2014 6:55pm': 'Happy birthday big boi! ',
 '01/20/2015 6:13am': 'Happy Birthday Bro! ',
 